In [ ]:
data_dir = '../../../Kaggle_Data/Salt/results/models_5/'
result_files = [
   ('FPN_224_Lz_relu0_lz_pseudo_f0_a684ac', None,0,4,2 ),# 0,1,2,3,4,5
    ('FPN_224_Lz_relu0_lz_pseudo_f1_43c64c', None,4,3,2  ),# 4, 3,2
    ('FPN_224_Lz_relu0_lz_pseudo_f2_74fccb', None,3,1,4  ),#3,1,4
    ('FPN_224_Lz_relu0_lz_pseudo_f3_1b3b85', None,3,1,4 ),#3,1,4
    ('FPN_224_Lz_relu0_lz_pseudo_f4_e61d36', None,1,4,5 ) #1,4,5,3
]
submission_file_name = data_dir+'FPN_224_Lz_relu0_lz_pseudo_f01234x4.flip-tmp'
save_results_dir = data_dir+'/../FPN_224_Lz_relu0_results/'
eval_crop_size = 224
use_flip = True

vote = False
vote_by_confidence = False

calc_val_test = (False, True)

In [ ]:
#threshold_best = np.mean(thresholds)
threshold_best = 0.5
threshold_best

In [ ]:
import numpy as np
import pandas as pd
import os
import load_data
from save_results import ResultsFileName, PredictResults

In [ ]:
if calc_val_test[0]:
    train_df = load_data.LoadData(train_data = True, DEV_MODE_RANGE = 0)
if calc_val_test[1]:
    test_df = load_data.LoadData(train_data = False, DEV_MODE_RANGE = 0)
    test_images = test_df.images

In [ ]:
total = 0
for fold, f_arr in enumerate(result_files):
    for j, x in enumerate(f_arr):
        if j == 0:
            params_file = x
        else:
            model = None
            for is_test_run in [False, True]:
                for flip in [False, True] :
                    if flip and not use_flip:
                        continue
                    if not calc_val_test[is_test_run]:
                        continue
                    data_file = ResultsFileName(save_results_dir, params_file, x, flip, is_test_run)
                    if not os.path.exists(data_file):
                        data_file += '.npy'
                    if os.path.exists(data_file):
                        print('loading '+ data_file)
                        r = np.load(data_file)
                    else:
                        print('creating ' + data_file)
                        if is_test_run:
                           images = test_images
                        else:
                           _, _, images, _ = load_data.SplitTrainData(train_df, fold)
                        assert False,'TEMPORARY BLOCKED'
                        r, model = PredictResults(model, images, data_dir, params_file, x, flip, is_test_run, save_results_dir, eval_crop_size = 224)
                    if is_test_run:
                        r = np.asarray(r)
                        if vote_by_confidence:
                            r_confidence = np.abs(r-0.5)
                            if total == 0:
                                s = r
                                s_confidence = r_confidence
                            else:
                                s = np.where(r_confidence>s_confidence, r, s)
                                s_confidence = np.where(r_confidence>s_confidence, r_confidence, s_confidence)
                        else:
                            if vote:
                                r = np.asarray(r>threshold_best, np.float32)
                            if total == 0:
                                s = r
                            else:
                                s += r
                        total += 1
                        print('done ', total)
if total and not vote_by_confidence:
    s /= total
    

In [ ]:
type(s), s.shape, type(r), r.shape

In [ ]:
def rle_encode(im):
    pixels = im.flatten(order = 'F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
if calc_val_test[1]:
    import time
    t1 = time.time()
    pred_dict = {idx: rle_encode((s[i] > threshold_best)) for i, idx in enumerate(test_df.index.values)}
    t2 = time.time()


In [ ]:
if calc_val_test[1]:
    sub = pd.DataFrame.from_dict(pred_dict,orient='index')
    sub.index.names = ['id']
    sub.columns = ['rle_mask']

In [ ]:
if calc_val_test[1]:
    submission_full_file_name = submission_file_name+'.submission.csv.gz'
    assert not os.path.exists(submission_full_file_name)
    sub.to_csv(submission_full_file_name, compression = 'gzip')
    print('saved: ', submission_full_file_name)